In [1]:
import sys  
sys.path.insert(0, '../AuxiliarCodes/')

In [2]:
from   gurobipy   import GRB
from globalFunctions import getNumberOfBlocksInADimension
from openPitFunctions import finalBlock
import gurobipy   as     gp
import pandas as pd
import re
import subprocess as sp
import random


In [3]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/" #Desktop
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'Modelo_F_OG_4_4_4.xlsx'

In [4]:
openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [5]:
#Falta agregar la restricción GQC_Up_OP y GQC_low_OP
#Dejar como indice, valor (formulario de función objetivo), duracion,tonelaje bloque, tonelaje * % Cu
#./omp.sh /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/files/openPit.* /home/williams/Tesis-Magister/ThesisCode/BendersCode/openPit/params/openPit.params
#./omp.sh ./openPit/files/openPit.* ./openPit/params/openPit.params

#1 periodo como defecto

In [14]:
class OpenPitModel:
   def __init__(self, database, numberOfPeriods):
      self.database = database
      self.numberOfPeriods = numberOfPeriods
      self.numberOfDestinations = 1
      self.basePrice = 3791.912
      self.desc = 0.1

   def execute(self):
      self.setOpenPitVariables()
      self.getOpenPitInfo()
      self.setOpenPitParameters()
      self.setOpenPitMineLimits()
      self.createOmpInput()
      return (self.executeOmp())
      #self.setFinalParameters()

   def setOpenPitVariables(self):
      self.openPitBlocksLength = self.database['X'].to_dict() 
      self.openPitBlocksWidth = self.database['Y'].to_dict() 
      self.openPitBlocksHeight = self.database['Z'].to_dict() #Los bloques se orientan de abajo hacia arriba, el bloque 0 es el que esta más abajo, 784 bloques
      self.L_b = self.database['Ton'].to_dict() #openPitBlockTonnage
      self.o_b = self.database['Mineral'].to_dict() #openPitBlockMineral
      self.openPitBlockRecovery = self.database['Recuperación'].to_dict() #openPitBlockRecovery
      self.openPitCopperLaw = self.database['%Cu'].to_dict() #openPitCopperLaw
      self.c_pbt = self.database['CPlanta CA'].to_dict() #openPitPlantCapacity
      self.c_mbt = self.database['CMina CA'].to_dict() #openPitMineCapacity

   def getOpenPitInfo(self):
      self.openPitBlocks = [i for i in range(len(self.openPitBlocksLength.values()))]

   def setOpenPitParameters(self):
      #OpenPit Parameters
      self.t_C   = {period : period + 1 for period in range(self.numberOfPeriods)}
      self.RMu_t = {period : 13219200.0 for period in range(self.numberOfPeriods)}#Superior infinita, 0 por abajo Originales: 13219200
      self.RMl_t = {period : 8812800.0 for period in range(self.numberOfPeriods)}#Valor original 8812800.0
      self.RPu_t = {period : 10933380.0 for period in range(self.numberOfPeriods)}#Valor original 10933380.0
      self.RPl_t = {period : 0 for period in range(self.numberOfPeriods)}#Valor original 7288920.0 
      self.qu_t  = {period : 1 for period in range(self.numberOfPeriods)}#Leyes promedio maxima y minima.
      self.ql_t  = {period : 0 for period in range(self.numberOfPeriods)}
      self.delta = {period: 0 for period in range(self.numberOfPeriods)}
      self.maxTimeOpenPit = self.t_C[max(self.t_C)]

   def setOpenPitMineLimits(self):
      self.openPitBlocksLengthLimits = getNumberOfBlocksInADimension(self.openPitBlocksLength)
      self.openPitBlocksWidthLimits = getNumberOfBlocksInADimension(self.openPitBlocksWidth)
      self.openPitBlocksHeightLimits = getNumberOfBlocksInADimension(self.openPitBlocksHeight)
      self.predecessorBlock = self.setPredecessorBlocks()

   def setModelAndGetResults(self):
      self.objValue, self.variableValues, self.runtime, self.gap = self.openPitModel()

   def setPredecessorBlocks(self):
      self.predecessorBlocks = finalBlock(self.openPitBlocks, self.openPitBlocksLengthLimits,self.openPitBlocksWidthLimits, self.openPitBlocksHeightLimits)

   def createOmpInput(self):
      self.writeProblemFile()
      self.writeBlocksFile()
      self.writePrecFile()
      self.writeParamsFile()
   
   def writeProblemFile(self):
      with open('../FilesToExecuteOmpOpenPit/files/openPit.prob', 'w') as f:
         numberOfDestinations = 'NDESTINATIONS: ' + str(self.numberOfDestinations)
         numberOfPeriods = 'NPERIODS: ' + str(self.numberOfPeriods)
         objective = 'OBJECTIVE: 0 1'
         duration = 'DURATION: 2'
         discountRate = 'DISCOUNT_RATE: '+ str(self.desc)#/self.numberOfPeriods)
         
         tonUpConstraint = 'CONSTRAINT: 0 3 P * L '
         for rmu in self.RMu_t.values():
            tonUpConstraint +=str(rmu) + " "

         tonLowContraint = 'CONSTRAINT: 1 3 P * G '
         for rml in self.RMl_t.values():
            tonLowContraint +=str(rml) + " "
         
         matUpConstraint = 'CONSTRAINT: 2 4 P * L '
         for rpu in self.RPu_t.values():
            matUpConstraint +=str(rpu) + " "

         matLowConstraint = 'CONSTRAINT: 3 4 P * G '
         for rpl in self.RPl_t.values():
            matLowConstraint +=str(rpl) + " "
         
         copperLawUpConstraint = 'CONSTRAINT: 4 5 P * L ' 
         for qut in self.qu_t.values():
            copperLawUpConstraint +=str(qut*sum(self.L_b.values())) + " "

         copperLawLowConstraint = 'CONSTRAINT: 5 5 P * G '
         for qlt in self.ql_t.values():
            copperLawLowConstraint +=str(qlt*sum(self.L_b.values())) + " "

         infeasibleBlocks = 'CONSTRAINT: 6 6 P * L '
         for delta in self.delta.values():
            infeasibleBlocks +=str(delta) + " "
         constraints = [tonUpConstraint, tonLowContraint, matUpConstraint, matLowConstraint, copperLawUpConstraint,copperLawLowConstraint,infeasibleBlocks]
         self.numberOfConstraints = len(constraints)
         self.nConstraints = 'NCONSTRAINTS: ' + str(self.numberOfConstraints)
         f.write('{}\n{}\n{}\n{}\n{}\n{}\n'.format(numberOfDestinations,numberOfPeriods, objective,duration,discountRate,self.nConstraints))
         f.write('{}\n{}\n{}\n{}\n{}\n{}\n{}\n'.format(*constraints))

   def writeBlocksFile(self):
      with open('../FilesToExecuteOmpOpenPit/files/openPit.blocks', 'w') as f:
         for block in self.openPitBlocks:
            index = block
            value = ((self.basePrice*self.openPitCopperLaw[block]-self.c_pbt[block])*self.o_b[block])-(self.c_mbt[block]*self.L_b[block])
            duration = 1
            ton = self.L_b[block]
            mineral = self.o_b[block]
            copperLaw = self.openPitCopperLaw[block] * self.L_b[block]
            #1 si no se puede extraer, 10977 última capa, los bloques van de abajo hacia arriba, 0 primer bloque de abajo, 10977 última capa hacia arriba, con 10975 la sol es vacia
            if block < 5902:
               f.write(('{} {} {} {} {} {} {}\n').format(index, value, duration,ton, mineral,copperLaw, 1))
            else:
               f.write(('{} {} {} {} {} {} {}\n').format(index, value, duration,ton, mineral,copperLaw, 0))
                 
   def writePrecFile(self):
      with open('../FilesToExecuteOmpOpenPit/files/openPit.prec', 'w') as f:
         for index,blockList in enumerate(self.predecessorBlocks):
            predecessorLine = str(len(blockList))
            for block in blockList:
               if block == index:
                  predecessorLine = " 0"
                  break
               else:
                  predecessorLine +=" " + str(block)
            f.write("{} {}\n".format(index, predecessorLine))
            
   def writeParamsFile(self):
      with open('../FilesToExecuteOmpOpenPit/params/openPit.params', 'w') as f:
         f.write("""USE_DISPLAY: 1
WRITE.LP.SOLUTION: 1
WRITE.IP.SOLUTION: 1
CPIT: 1
PP.ULTIMATE_PIT: 1
PP.FORCE_UPIT: 1
PP.EARLY_START: 1
PP.WASTE_OPTION: 1
PP.ELIM_NULL: 0
PP.TRANSITIVE_REDUCTION: 0
AG.USE_BLOCK_AGGREGATION: 0
AG.BLOCK_AGGREGATION: 0
OPTMETHOD: 0
CG.ONE_DESTINATION: 1
CG.IMPLICIT: 0
CG.USE_DISPLAY: 1
CG.MAX_ITER: -1
CG.TARGET_GAP: 0.0001
CG.MAX_TIME: -1
CG.USE_KSTEP: 0
CG.KSTEP_K: 10
CG.MASTER_NTHREADS: 4
CG.DISPLAY_DUALS: 1
HE.TOPOSORT: 1
HE.FTOPOSORT: 0
HE.NALPHA_POINTS: 50
HE.OPT_DESTINATIONS: 1
HE.NAIVE: 0
HE.NAIVE_INTSOLLIM: 1000
HE.NAIVE_EPGAP: 0.01
HE.NAIVE_TILIM: 14400
CP.DYNAMIC_CUTS: 0
CP.CLIQUES: 0
CP.MINW: 0
CP.DELAYED_PRECEDENCES: 0
CONSTRAINT_PROGRAMMING: 0
CPROG.GAP_LIMIT: 0.01
CPROG.CP_TIME_LIMIT: 28800
CPROG.EX_TIME_LIMIT: 28800
CPROG.HOT_START: 1
CPROG.NTHREADS: 8""")

   def executeOmp(self):
      output = sp.getoutput("./omp.sh ../FilesToExecuteOmpOpenPit/files/openPit.* ../FilesToExecuteOmpOpenPit/params/dbs_duals.params")
      print(output)
      return(output)
      #return self.getPiAndObjectiveValue(output)
   
   def getPiAndObjectiveValue(self, output):
      pi_positions = [positions.start() for positions in re.finditer("rhs= 0.000000", output)]
      pi_t = dict.fromkeys(self.t_C,0)
      for pos in pi_positions:
         pi_value = float(output[pos-48: pos].split()[-3])
         pi_index = float(output[pos-48: pos].split()[-5])
         period_index = pi_index - self.numberOfPeriods * (self.numberOfConstraints - 1)
         pi_t[period_index] = pi_value

      objective_value_positions = [positions.start() for positions in re.finditer("Objective Value ", output)]
      objective_value = float(output[objective_value_positions[0]: objective_value_positions[0]+100].split()[-3])

      return objective_value, pi_t 

In [15]:
numberOfPeriods = 5
openPitModel = OpenPitModel(openPitDataframe,numberOfPeriods)
#pi_vb, obective_value = openPitModel.execute()
#pi_vb, obective_value 
output = openPitModel.execute()


Running OMP version: master 1.8.5 d7d16093 2022-11-16 20:03:46 -0300
Invoked using: bin/OMP_SOLVER_C1290 ../FilesToExecuteOmpOpenPit/files/openPit.blocks ../FilesToExecuteOmpOpenPit/files/openPit.prec ../FilesToExecuteOmpOpenPit/files/openPit.prob ../FilesToExecuteOmpOpenPit/params/dbs_duals.params
license to sign [DEFAULTclient-signatureAFhG-4bsA-AAA=lic_ver200valid-to2023-03-07]
LICENSE OK!
BLOCKS FILE: ../FilesToExecuteOmpOpenPit/files/openPit.blocks
PREC FILE: ../FilesToExecuteOmpOpenPit/files/openPit.prec
PROB FILE: ../FilesToExecuteOmpOpenPit/files/openPit.prob
PARAMS FILE: ../FilesToExecuteOmpOpenPit/params/dbs_duals.params
OR Solver: key created. The key is: openPit_openPit_dbs_duals
STATS_INPUT_NAME openPit_openPit_dbs_duals
OR Solver: loaded params (params file) OK.
STATS_PARAM_STOCHASTIC_MODE 0
Objective: NPV
DURATION loaded from column 2.
Loaded prob file OK.
Loaded block file columns OK.
Vars and Prob populated OK.
Block memory allocated OK.
Block information in VARS popul

In [8]:
with open('fifthScenarioSolution', 'w') as f:
        f.write(str(openPitModel.execute()))

../omp.sh: line 11: bin/OMP_SOLVER_C1290: No such file or directory


In [9]:
import re

pi_positions = [positions.start() for positions in re.finditer("rhs= 0.000000", output)]
pi_t = dict.fromkeys(openPitModel.t_C,0)
for pos in pi_positions:
    pi_value = float(output[pos-48: pos].split()[-3])
    pi_index = float(output[pos-48: pos].split()[-5])
    period_index = pi_index - openPitModel.numberOfPeriods * (openPitModel.numberOfConstraints - 1)
    pi_t[period_index] = pi_value

objective_value_positions = [positions.start() for positions in re.finditer("Objective Value ", output)]
objective_value = float(output[objective_value_positions[0]: objective_value_positions[0]+100].split()[-3])

objective_value, pi_t

IndexError: list index out of range

In [ ]:
output[objective_value_positions[0]: objective_value_positions[0]+100].split()

['Objective',
 'Value',
 '|',
 '6.7539e+08',
 '|',
 '6.7539e+08',
 '|',
 '|----------------------------']

In [ ]:
a = output[pos-48: pos].split()
b = int(a[-5])
a

['BZ_DUAL:', 'row=', '34', 'pi=', '5662813.786159', 'sense=', 'L']

In [ ]:
index


NameError: name 'index' is not defined

In [ ]:
pi_t

{0: 0, 1: 0, 2: 0, 3: 0, 4: 5662813.786159}

In [ ]:
a = numberOfPeriods * (7 - 1)#30
b = (numberOfPeriods * 7) -1#34

b-a

4